### HTML 데이터 파싱을 이용한 크롤링
- 네이버 연관 검색어
    - 1. 웹페이지 분석 : URL 찾기
    - 2. 요청 -> 응답 : HTML(str)
    - 3. HTML(str) -> BeautifulSoup obj -> DF

In [22]:
# 1. 웹페이지 분석 : URL
query = "삼성전자"
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query={}".format(query)
print(url)

https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=삼성전자


In [3]:
# 2. request -> response : HTML(str)
response = requests.get(url)
response

<Response [200]>

In [7]:
# 3. HTML(str) -> BeautifulSoup obj (css-selector) -> DF
dom = BeautifulSoup(response.content, "html.parser")
type(dom)

bs4.BeautifulSoup

In [ ]:
# 자주 사용하는 bs4 함수
# select : 여러개의 element 객체를 리스트로 가져옴
# select_one : 하나의 element 객체를 가져옴

In [10]:
# chrome에서 자동으로 selector 찾기
#nx_related_keywords > dl > dd.lst_relate._related_keyword_list > ul

elements = dom.select("._related_keyword_ul > li")
len(elements)

10

In [16]:
type(elements), elements[0]

(bs4.element.ResultSet,
 <li> <a data-area="*q" data-idx="1" href="?where=nexearch&amp;query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%9A%B0&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">삼성전자우</a> </li>)

In [20]:
keywords = []
for element in elements:
    keywords.append(element.text.strip())

print(keywords)

['삼성전자우', '카카오', 'sk하이닉스', '삼성', '네이버', '현대자동차', '삼성전자서비스센터', 'lg화학', '현대차', '코스피']


In [21]:
keywords = [element.text.strip() for element in elements]
print(keywords)

['삼성전자우', '카카오', 'sk하이닉스', '삼성', '네이버', '현대자동차', '삼성전자서비스센터', 'lg화학', '현대차', '코스피']


In [23]:
# 함수로 만들기
def naver_related_keywords(query):
    url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query={}".format(query)
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select("._related_keyword_ul > li")
    return [element.text.strip() for element in elements]

In [25]:
query = "코로나"
result = naver_related_keywords(query)
print(result)

['코로나 3단계', '코로나 라이브', '코로나 2.5단계', '코로나 확진자', '코로나 2.5단계 연장', '태풍', '코로나 백신', '코로나 3단계 격상', '날씨', '사회적 거리두기 3단계']


#### 다음 뉴스 데이터
- 다음 뉴스의 메인 페이지 15개 기사의 제목, 링크, 내용 수집

In [30]:
# 1. 웹페이지 분석 : URL
url = "https://news.daum.net/"

In [31]:
# 2. request -> response : HTML(str)
response = requests.get(url)
response

<Response [200]>

In [32]:
# 3. HTML(str) -> BeautifulSoup obj (css-selector) -> DF
dom = BeautifulSoup(response.content, "html.parser")
dom


<!DOCTYPE html>

<html class="os_unknown none unknown version_0" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="always" name="referrer">
<meta content="Daum 뉴스" property="og:author"/>
<meta content="다음뉴스" property="og:site_name"/>
<meta content="홈" property="og:title"/>
<meta content="https://t1.daumcdn.net/media/img-media/mobile/meta/news.png" property="og:image"/>
<meta content="다음뉴스" property="og:description"/>
<link href="https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon"/>
<title>홈 | 다음뉴스</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="//t1.daumcdn.net/media/kraken/news/0da6ce5/common.css.merged.css" rel="stylesheet" type="text/css">
<link href="//t1.daumcdn.net/media/kraken/news/0da6ce5/news.css.merged.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/media/kraken/news/0da6ce5/calendar.css.merged.css" rel="stylesheet" type="text/css"/>
<!--[if lte IE 8]>
<script src="https://m2.daumcdn.net/svc/ori

In [33]:
elements = dom.select(".list_headline > li")
len(elements)

15

In [34]:
type(elements), elements[0]

(bs4.element.ResultSet,
 <li class="item_main">
 <a class="link_cont" data-tiara-custom="contentUniqueKey=hamny-20200905211808398" data-tiara-id="20200905211808398" data-tiara-layer="photo" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://news.v.daum.net/v/20200905211808398">
 <span class="wrap_thumb">
 <img '코로나="" 90%="" alt="좀 나아지나 했더니.." class="thumb_g" src="https://img1.daumcdn.net/thumb/S189x104ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202009%2F05%2Fkbs%2F20200905211809482rqbx.jpg&amp;scode=media" 곤두박질""="" 매출="" 재확산'=""/>
 </span>
 <strong class="tit_g">좀 나아지나 했더니.."'코로나 재확산' 매출 90% 곤두박질"</strong>
 </a>
 </li>)

In [37]:
data = []
for element in elements:
    data.append({
        "title": element.select_one(".tit_g").text.strip().replace("\n", ""),
        "link": element.select_one("a").get("href"),
    })

articles_df = pd.DataFrame(data)
articles_df.tail(2)

,title,link
13,러시아서 코로나19 5205명 추가 감염..누적 102만명 넘어뉴시스,https://news.v.daum.net/v/20200905204951081
14,"정부-의협 합의에도..업무 복귀 않는 전공의들, 왜?SBS",https://news.v.daum.net/v/20200905204209960


In [38]:
# content 데이터 추가

In [56]:
link = articles_df.loc[14, "link"]
print(link)

https://news.v.daum.net/v/20200905204209960


In [42]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

In [46]:
content = dom.select_one("#harmonyContainer").text.strip().replace("\n", "")
content[:100]

'[앵커]코로나19 재확산으로 강도 높은 거리두기가 연장되면서 소상공인들 시름도 더욱 깊어지고 있습니다.언제까지 버틸 수 있을지 모르겠단 호소가 여기저기서 나오는데요. \u200b하루하루 힘'

In [47]:
# 기사 요약 부분이 있으면 이것도 가능
content = dom.select_one(".summary_view").text.strip().replace("\n", "")
content[:100]

AttributeError: 'NoneType' object has no attribute 'text'

In [48]:
def get_content(link):
    response = requests.get(link)
    dom = BeautifulSoup(response.content, "html.parser")
    return dom.select_one("#harmonyContainer").text.strip().replace("\n", "")

In [53]:
articles_df["contents"] = articles_df["link"].apply(get_content)
articles_df.tail()

,title,link,contents
10,'초등학생 가구에 돌봄쿠폰 지급' 유력 검토,https://news.v.daum.net/v/20200905211744390,[앵커]2차 재난지원금은 1차 때와 달리 경제적인 타격이 큰 계층에 선별 지급하겠다...
11,남양주 아동도서업체 관련 3명 추가 확진..누적 11명연합뉴스,https://news.v.daum.net/v/20200905205603153,(남양주=연합뉴스) 김도윤 기자 = 경기 남양주시는 5일 오전 진접읍에 사는 A(남...
12,"""수십 명 인명 피해 나""..북한, '대비 소홀' 간부 엄벌SBS",https://news.v.daum.net/v/20200905205103096,<앵커>9호 태풍 '마이삭'으로 북한 강원도 지역이 물바다가 돼서 수십 명의 인명 ...
13,러시아서 코로나19 5205명 추가 감염..누적 102만명 넘어뉴시스,https://news.v.daum.net/v/20200905204951081,[모스크바=AP/뉴시스] 러시아는 11일(현지시간) 신종 코로나바이러스 감염증(코로...
14,"정부-의협 합의에도..업무 복귀 않는 전공의들, 왜?SBS",https://news.v.daum.net/v/20200905204209960,"<앵커>병원에서 환자를 가장 일선에서 보는 이 인턴, 레지던트들, 전공의들이 돌아와..."
